In [10]:
import shutil
import os

In [11]:
pwd = os.getcwd()
#pwd += "/rheo_bench/channel/"
pwd

'/home/epalermo/test/rheo_bench/dam_break'

# Generate LAMMPS Input Scripts

In [17]:
#constants
hd = "3.5"
eta = "0.1"
Dr = "0.1"
fext = "1e-5"
cutoff = "25"
shifting = "shift"
surface = "fix                3 all rheo/surface ${h3} 1.5 "+cutoff
sf = "0.1"


#parameter grids
var_surface_grid = {"fix                2 all rheo/surface ${h3} 1.5 "+cutoff : "surface_on",
                    "":"surface_off"}
var_sf_grid = {"0.05":"20","0.1":"10","0.2":"5","0.25":"4","0.5":"2"}
var_kernel_grid = {"CRK0":"0","CRK1":"1","CRK2":"2","Quintic":"5"}
var_shifting_grid = {"shift":"shift_on","":"shift_off"}
var_eta_grid = {"0.1":"0p1"}



In [20]:
#Generate lammps files

out_filepath = "static"

filenames = []
#for surface in var_surface_grid:
#    for shifting in var_shifting_grid:
#        for sf in var_sf_grid:
for kernel in var_kernel_grid:
    run_code = var_kernel_grid[kernel]+"_"+var_sf_grid[sf]+"_"+var_eta_grid[eta]+"_"+fext
    filename = "lammps_in__"+run_code
    file = shutil.copyfile("lammps_in.template",pwd+"/"+filename)
    file_in = open(file,"rt")
    file_text = file_in.read()

    #Replace variables
    file_text = file_text.replace("var_sf",sf)
    file_text = file_text.replace("var_hd",hd)
    file_text = file_text.replace("var_Dr",Dr)
    file_text = file_text.replace("var_fext",fext)
    file_text = file_text.replace("var_eta",eta)
    file_text = file_text.replace("var_kernel",kernel)
    file_text = file_text.replace("var_cutoff",cutoff)
    file_text = file_text.replace("var_shift",shifting)
    file_text = file_text.replace("var_surface",surface)
    file_text = file_text.replace("var_ncdf_path",out_filepath+"/ncdf/"+run_code)
    file_text = file_text.replace("var_dump_path",out_filepath+"/dump/"+run_code)

    file_out = open(file,"wt")
    file_out.write(file_text)
    
    filenames.append(filename)

print(filenames)

['lammps_in__0_10_0p1_1e-5', 'lammps_in__1_10_0p1_1e-5', 'lammps_in__2_10_0p1_1e-5', 'lammps_in__5_10_0p1_1e-5']


# Write the Solo Jobscript

In [ ]:
account="FY150028"
job_name="TestChannel"
time="24:0:0"
partition="batch"
nodes="1"
ntasks_per_node="16"
run_command = "mpirun -np 16 /ascldap/users/etpaler/dev/lammps_rheo/lammps/build/lmp -in "

In [ ]:
with open("jobscript","w") as jobscript:
        #---shebang---#
        jobscript.write("#!/bin/bash\n\n")

        #---slurm commands---#
        jobscript.write("#SBATCH --account={}\n".format(account))
        jobscript.write("#SBATCH --job-name={}\n".format(job_name))
        jobscript.write("#SBATCH --time={}\n".format(time))
        jobscript.write("#SBATCH --partition={}\n".format(partition))
        jobscript.write("#SBATCH --nodes={}\n".format(nodes))
        jobscript.write("#SBATCH --ntasks-per-node={}\n\n".format(ntasks_per_node))
        
         #---unix commands---#
        jobscript.write("module load cmake/3.20.3 intel/18.0.0.128 openmpi-intel/4.1 mkl/18.0.0.128\n\n")
        
        for filename in filenames:
            jobscript.write(run_command+filename+"\n")
            
        jobscript.write("\n")

# Write the WSL Jobscript

In [9]:
run_command = "mpirun -np 4 ~/dev/rlammps_wall_debug/build/lmp -in "

with open("jobscript","w") as jobscript:
        #---shebang---#
        jobscript.write("#!/bin/bash\n\n")

        for filename in filenames:
            jobscript.write(run_command+filename+"\n")
            
        jobscript.write("\n")